In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
%matplotlib inline

In [69]:
combined_df = pd.read_csv('combined_binary_df.csv', index_col='date', infer_datetime_format=True, parse_dates=True)

In [70]:
X = combined_df.drop(columns = 'class')
y = combined_df['class'].to_frame()
# standardize X values
X = (X - X.mean())/X.std()

In [71]:
# define functions to change dataframe to array
def target(df, column, steps):
    target = []
    for i in range(len(df) - steps):
        t = df.iloc[i + steps - 1, column]
        target.append(t)
    return np.array(target).reshape(-1, 1)

def features(df, columns, steps):
    features = []
    for c in range(len(columns)):
        feature = []
        for i in range(len(df) - steps):
            f = list(df.iloc[i : i + steps, columns[c]])
            feature.append(f)
        features.append(feature)
    return np.array(features).T

In [72]:
X = features(X, list(range(8)), 10)
y = target(y, 0, 10)

In [73]:
X.shape

(10, 1440, 8)

In [74]:
y.shape

(1440, 1)

In [75]:
X = X.reshape(1440, 10, 8)

In [76]:
tt_split = int(len(X) * 0.8)
X_train = X[: tt_split]
X_test = X[tt_split :]
y_train = y[: tt_split]
y_test = y[tt_split :]

In [77]:
# one-hot-encode y values
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [78]:
model = Sequential()
neurons = 30
model.add(LSTM(units=neurons, return_sequences=True, input_shape=(X.shape[1], 8)))
model.add(Dropout(0.2))
model.add(LSTM(units=neurons))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))

In [79]:
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [80]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 10, 30)            4680      
                                                                 
 dropout_8 (Dropout)         (None, 10, 30)            0         
                                                                 
 lstm_9 (LSTM)               (None, 30)                7320      
                                                                 
 dropout_9 (Dropout)         (None, 30)                0         
                                                                 
 dense_4 (Dense)             (None, 2)                 62        
                                                                 
Total params: 12,062
Trainable params: 12,062
Non-trainable params: 0
_________________________________________________________________


In [83]:
# define earliystopping
stop = EarlyStopping(monitor='accuracy', mode='min', patience=10, verbose=1)

In [84]:
model.fit(X_train, y_train, epochs=100, batch_size=5, verbose=1, shuffle=False, callbacks=[stop])

Epoch 1/100
231/231 [==============================] - 1s 4ms/step - loss: 0.6425 - accuracy: 0.6181
Epoch 2/100
231/231 [==============================] - 1s 4ms/step - loss: 0.6310 - accuracy: 0.6380
Epoch 3/100
231/231 [==============================] - 1s 4ms/step - loss: 0.6244 - accuracy: 0.6337
Epoch 4/100
231/231 [==============================] - 1s 4ms/step - loss: 0.6195 - accuracy: 0.6510
Epoch 5/100
231/231 [==============================] - 1s 4ms/step - loss: 0.6111 - accuracy: 0.6476
Epoch 6/100
231/231 [==============================] - 1s 4ms/step - loss: 0.6100 - accuracy: 0.6580
Epoch 7/100
231/231 [==============================] - 1s 4ms/step - loss: 0.6019 - accuracy: 0.6632
Epoch 8/100
231/231 [==============================] - 1s 5ms/step - loss: 0.5871 - accuracy: 0.6814
Epoch 9/100
231/231 [==============================] - 1s 4ms/step - loss: 0.5843 - accuracy: 0.6658
Epoch 10/100
231/231 [==============================] - 1s 5ms/step - loss: 0.5640 - accura

In [85]:
score = model.evaluate(X_test, y_test)[1]
print("%0.2f accuracy" % (score))

9/9 [==============================] - 1s 2ms/step - loss: 0.8481 - accuracy: 0.5104
0.51 accuracy
